
### This notebook is used to convert ROI on single slices of the tiff-stacks into contigous detections.

In [1]:
%pylab inline
import numpy as np
import sys
sys.path.append('../src')
import det_sort as ds
from convert_ROI_to_cell_volumes import *
from ijroi.ij_roi import Roi
from ijroi.ijpython_encoder import encode_ij_roi,RGB_encoder
from ijroi.ijpython_decoder import decode_ij_roi
from find_maxima import find_maxima
import tifffile
import glob
import skimage.io as io

from scipy import ndimage
import pyperclip

def find_local_maxima_np(img_data):
    #This is the numpy/scipy version of the above function (find local maxima).
    #Its a bit faster, and more compact code.
    
    #Filter data with maximum filter to find maximum filter response in each neighbourhood
    max_out = ndimage.filters.maximum_filter(img_data,size=3)
    #Find local maxima.
    local_max = np.zeros((img_data.shape))
    local_max[max_out == img_data] = 1
    local_max[img_data == np.min(img_data)] = 0
    return local_max.astype(np.bool)

def return_maxima(img_vol,ntol):
    cell_shape = img_vol.__len__()
    img = img_vol[cell_shape//2]
    
    find_local_maxima_np(img)
    
    img_data = np.array(img).astype(np.float64)
    #Finds the local maxima using maximum filter.
    local_max = find_local_maxima_np(img_data)
    y,x,out = find_maxima(img_data,local_max.astype(np.uint8),ntol)
    return y,x


Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


### Conversion of ROI to contiguous cell regions

In [2]:
###Parameters.

img_width = 512 #pixels
img_height = 512 #pixels
border_offset = 12 #If regions end or start within this they are included.
scale = 0.26 #um/pixel
zspacing = 0.50 #physical distance (um) between z-slices.
filepaths = []
outpaths = []


#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0002_2020_11_20_high/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0003_2020_11_23_low/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0004_2020_11_23_high/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0005_2020_11_27_low/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0006_2020_11_27_high/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0007_2020_11_30_low/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0008_2020_11_30_high/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0009_2020_12_04_low/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0010_2020_12_04_high/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0011_2020_12_07_low/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0012_2020_12_07_high/")

#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0013_2020_12_12_low/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0014_2020_12_12_high/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0015_2020_12_14_low/")
#filepaths.append("/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0016_2020_12_14_high/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel05/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel06/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel07/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel08/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel09/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel10/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel11/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel12/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel13/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel14/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel15/")
filepaths.append("/Users/dominicwaithe/Desktop/relabel16/")

#outpaths.append("/Users/dominicwaithe/Desktop/out02/")
#outpaths.append("/Users/dominicwaithe/Desktop/out03/")
#outpaths.append("/Users/dominicwaithe/Desktop/out04/")
#outpaths.append("/Users/dominicwaithe/Desktop/out05/")
#outpaths.append("/Users/dominicwaithe/Desktop/out06/")
#outpaths.append("/Users/dominicwaithe/Desktop/out07/")
#outpaths.append("/Users/dominicwaithe/Desktop/out08/")
#outpaths.append("/Users/dominicwaithe/Desktop/out09/")
#outpaths.append("/Users/dominicwaithe/Desktop/out10/")
#outpaths.append("/Users/dominicwaithe/Desktop/out11/")
#outpaths.append("/Users/dominicwaithe/Desktop/out12/")
#outpaths.append("/Users/dominicwaithe/Desktop/out13/")
#outpaths.append("/Users/dominicwaithe/Desktop/out14/")
#outpaths.append("/Users/dominicwaithe/Desktop/out15/")
#outpaths.append("/Users/dominicwaithe/Desktop/out16/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel05out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel06out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel07out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel08out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel09out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel10out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel11out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel12out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel13out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel14out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel15out/")
outpaths.append("/Users/dominicwaithe/Desktop/relabel16out/")



                      

In [3]:
for filepath, outpath in zip(filepaths,outpaths):
    convObj = ConvertROItoCellVolumes(img_width, img_height, border_offset, scale, zspacing, filepath, outpath)
    convObj.append_new_regions(outpath, extend_roi=False)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


### Rudimentary Analysis Example
Measuring mean intensity of cell volumes.

In [3]:

channel = 0
cell_data = []
for path in outpaths:
    cell_data.append(collect_info(path,channel,'mean'))
copy_to_clipboard(cell_data)

no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays present in file.
no Overlays pr

### Save out image 

In [ ]:
channel = 0
cell_data = []
img_path = '/Users/dominicwaithe/Desktop/cell_images'
imgnum = 0
for path in outpaths:
    data = collect_info(path,channel,'raw')
    store_cell_data = []
    for cell in range(0,data.__len__()):
        img_vol = data[cell]
        cell_shape = img_vol.__len__()
        raw_img = img_vol[cell_shape//2]
        img = normalise_for_8bit(raw_img)
        n = str(imgnum)
        io.imsave(img_path+"/"+n.zfill(8)+".png", img)  
        imgnum +=1

### Custom analysis - count foci

In [ ]:
ntol = 500 #Noise Tolerance.
channel = 1
cell_data = []

for path in outpaths:
    data = collect_info(path,channel,'raw')
    store_cell_data = []
    for cell in range(0,data.__len__()):
            y,x = return_maxima(data[cell],ntol)
            store_cell_data.append(y.__len__())
    cell_data.append(store_cell_data)

copy_to_clipboard(cell_data)

In [ ]:
img.shape

In [ ]:
filepaths

In [ ]:
tfile = tifffile.TiffFile('/Users/dominicwaithe/Documents/collaborators/SwiftLonnie/screens/acquisitions/0004_2020_11_23_high/images/img_stk_x_-10004.7y_1740.8.tif')

In [ ]:
strn = tfile.imagej_metadata['Info']
lines = strn.split("\n")
for entry in lines:
    if entry[:12] == 'Stage X-pos:':
        stagex = float(entry.split(":")[1].split("um.")[0])
    if entry[:12] == 'Stage Y-pos:':
        stagey = float(entry.split(":")[1].split("um.")[0])
    if entry[:11] == 'XY-spacing:':
        voxelxy = float(entry.split(":")[1].split("um, Z-spacing")[0])
    if entry[:12] == 'Piezo Z-pos:':
        stagez = np.array(entry.split(":")[1].split(", (um)")[0].split(',')).astype(np.float64)

In [ ]:
stagez.sort()
print( stagez)

In [ ]:
tfile.imagej_metadata